# Import required libraries

In [ ]:
import torch
import torch.nn as nn

from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

# Load Dataset

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])

train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


# Load and define the model


In [ ]:
!pip install timm

In [ ]:
vit_model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)

num_labels = 10
vit_model.head = nn.Linear(vit_model.head.in_features, num_labels)

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main
/root/.cache/torch/hub/facebookresearch_deit_main/models.py:63: UserWarning: Overwriting deit_tiny_patch16_224 in registry with models.deit_tiny_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def deit_tiny_patch16_224(pretrained=False, **kwargs):
/root/.cache/torch/hub/facebookresearch_deit_main/models.py:78: UserWarning: Overwriting deit_small_patch16_224 in registry with models.deit_small_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def deit_small_patch16_224(pretrained=False, **kwargs):
/root/.cache/torch/hub/facebookresearch_deit_main/models.py:93: UserWarning: Overwriting deit_base_patch16_224 in registry with models.deit_base_patch16_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.

In [ ]:
# vit_model.parameters()

In [ ]:
for param in vit_model.parameters():
    param.requires_grad = False

for param in vit_model.head.parameters():
    param.requires_grad = True

# Define the training configurations

In [ ]:
writer = SummaryWriter()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters(), lr=0.001)

num_epochs = 2

# Train the model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model.to(device)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [ ]:
train_dataset[0][0].shape

torch.Size([3, 224, 224])

In [ ]:
from tqdm import tqdm

In [ ]:
len(train_dataset), len(train_loader)

(60000, 938)

In [ ]:
for epoch in range(num_epochs):
  vit_model.train()
  running_loss = 0
  correct_train = 0
  total_train = 0

  for i, (inputs, labels) in tqdm(enumerate(train_loader)):
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    # print(inputs.shape)
    outputs = vit_model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    _, predicted = torch.max(outputs, 1)
    correct_train += (predicted == labels).sum().item()
    total_train += labels.size(0)

  epoch_loss = running_loss / len(train_loader)
  train_accuracy = correct_train / total_train
  print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Training Accuracy: {100 * train_accuracy:.2f}%")

  writer.add_scalar('Loss/train', epoch_loss, epoch)
  writer.add_scalar('Accuracy/train', train_accuracy, epoch)

writer.close()

938it [12:30,  1.25it/s]


Epoch 1/2, Loss: 0.4087, Training Accuracy: 86.33%


938it [12:29,  1.25it/s]

Epoch 2/2, Loss: 0.2935, Training Accuracy: 89.54%


In [ ]:
vit_model.eval()

correct_test = 0
total_test = 0

with torch.no_grad():
  for inputs, labels in tqdm(test_loader):
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = vit_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total_test += labels.size(0)
    correct_test += (predicted == labels).sum().item()

test_accuracy = correct_test / total_test
print(f"Test Accuracy: {100 * test_accuracy:.2f}%")

100%|██████████| 157/157 [02:05<00:00,  1.25it/s]

Test Accuracy: 89.10%


# Onnx Export

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install onnx
!pip install onnxscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.8/594.8 kB 12.9 MB/s eta 0:00:00


In [ ]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)
onnx_program = torch.onnx.dynamo_export(vit_model, dummy_input)

onnx_program.save("/content/drive/MyDrive/dlops/vit_model_fashionmnist.onnx")

/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


# Import onnx model

In [ ]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 15.8 MB/s eta 0:00:00


In [ ]:
import onnxruntime
import numpy as np

onnx_model_path = "/content/drive/MyDrive/dlops/vit_model_fashionmnist.onnx"
ort_session = onnxruntime.InferenceSession(onnx_model_path)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)
inputs, labels = next(iter(test_loader))
input_tensor = inputs.numpy()

ort_inputs = {ort_session.get_inputs()[0].name: input_tensor}
ort_outputs = ort_session.run(None, ort_inputs)

predicted_label_index = np.argmax(ort_outputs[0])
predicted_label = test_dataset.classes[predicted_label_index]

print(f"Predicted Label: {predicted_label}")

Predicted Label: Trouser


# Save the traced model

In [ ]:
vit_model = vit_model.to('cpu')

In [ ]:
sample_input = torch.randn(1, 3, 224, 224)
traced_model = torch.jit.trace(vit_model, sample_input)

traced_model.save("/content/drive/MyDrive/dlops/fashion_mnist_vit_scripted_cpu.pt")

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1499: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
